<a href="https://colab.research.google.com/github/Atharva-Phatak/NLP-Project-LU/blob/khande/bert_lstm_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qq plotly
!pip install -qq icecream
!pip install -qq transformers

     |████████████████████████████████| 2.5MB 5.1MB/s 
     |████████████████████████████████| 3.3MB 16.2MB/s 
     |████████████████████████████████| 901kB 22.0MB/s 


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import numpy as np
import random as rn
from icecream import ic 
from torch.utils.data import Dataset, DataLoader
import os

In [ ]:
# Initializing
train_count = 8000 #8000  
MAX_SENTENCE_LENGTH = 128
MAX_SENTENCES = 5
MAX_LENGTH = 100
BASENET = 'bert-base-uncased'

In [ ]:
# SELECT DEVICE CPU/CUDA
cuda_available = torch.cuda.is_available()
device   = torch.device("cuda") if (cuda_available) else torch.device("cpu")

seed=1
torch.manual_seed(seed)
if(cuda_available):
	torch.cuda.manual_seed(seed)
	torch.backends.cudnn.benchmark = True
np.random.seed(seed)

ic(torch.cuda.is_available(), device, seed)

ic| torch.cuda.is_available(): False
    device: device(type='cpu')
    seed: 1


(False, device(type='cpu'), 1)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# LOAD DATA
!pwd
%cd /content/drive/MyDrive/Colab\ Notebooks/Spring\ 21/NLP/Humour-Detection-2021
train_df = pd.read_csv("data/train.csv")
display(train_df.count())
display(train_df[:1])
ic(train_df['text'][0])

/content
/content/drive/MyDrive/Colab Notebooks/Spring 21/NLP/Humour-Detection-2021


id                   8000
text                 8000
is_humor             8000
humor_rating         4932
humor_controversy    4932
offense_rating       8000
dtype: int64

,id,text,is_humor,humor_rating,humor_controversy,offense_rating
0,1,TENNESSEE: We're the best state. Nobody even c...,1,2.42,1.0,0.2


ic| train_df['text'][0]: ("TENNESSEE: We're the best state. Nobody even comes close. *Elevennessee "
                          'walks into the room* TENNESSEE: Oh shit...')


"TENNESSEE: We're the best state. Nobody even comes close. *Elevennessee walks into the room* TENNESSEE: Oh shit..."

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(BASENET,
              add_special_tokens= True, # [CLS] [SEP]
							 max_length= 128, 
							 padding= 'max_length', 
							 return_attention_mask= True,
               return_tensors = 'pt',
							 truncation= True)

In [ ]:
encoded_input = tokenizer(train_df['text'][5])

ic(torch.is_tensor(encoded_input['input_ids']))
print('input_ids : ',encoded_input['input_ids'])
print('token_type_ids : ',encoded_input['token_type_ids'])
print(encoded_input['attention_mask'])
ic(len(encoded_input['input_ids']))  #len keeps changing depending on text length
ic(tokenizer.decode(encoded_input['input_ids']))


ic| torch.is_tensor(encoded_input['input_ids']): False
ic| len(encoded_input['input_ids']): 29


[101, 1005, 19817, 19736, 5558, 1010, 1005, 1996, 3009, 2773, 2005, 2147, 1010, 3310, 2013, 1996, 3763, 2744, 1005, 29461, 12952, 2401, 2890, 1010, 1005, 3574, 8639, 1012, 102]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


ic| tokenizer.decode(encoded_input['input_ids']): ("[CLS]'trabajo,'the spanish word for work, comes from the latin "
                                                   "term'trepaliare,'meaning torture. [SEP]")


"[CLS]'trabajo,'the spanish word for work, comes from the latin term'trepaliare,'meaning torture. [SEP]"

In [ ]:
# 
llm_model = AutoModel.from_pretrained( BASENET, 
                                           output_hidden_states = False)
seg_id = torch.tensor([1] * 128).view(1, -1)

embedding = llm_model (input_ids=encoded_input['input_ids'], 
                       attention_mask= encoded_input['attention_mask'],
                     token_type_ids=seg_id)
ic(embedding.shape)

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
# CUSTOM DATASET CLASS
class MyCustomDataset(Dataset): #dataset from torch
    def __init__(self, filename, basenet= 'bert-base-uncased', max_len= 128):
        super(MyCustomDataset, self).__init__()
        self.data    = pd.read_csv(filename) #"data/train.csv"
        self.labels = self.data['is_humor']
        self.text = self.data['text']
        self.tokenizer = AutoTokenizer.from_pretrained(basenet)
        self.max_len = max_len
        self.segment_id = torch.tensor([1] * self.max_len).view(1, -1)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        label = self.labels[idx]
        text = self.text[idx]

        label = torch.tensor(label)

        encoded_input = self.tokenizer(text= text, # sentence
							 add_special_tokens= True, # [CLS] [SEP]
							 max_length= self.max_len, 
							 padding= 'max_length', 
							 return_attention_mask= True,
							 return_tensors = 'pt',  # return torch tensors
							 truncation= True
							)
        item = {"input_id": encoded_input['input_ids'], 
                "attention_mask_id": encoded_input['attention_mask'],
                "segment_id": self.segment_id,
                "true_label": label
              }
        
        return item 

In [ ]:
#MODEL ARCHITECTURE CLASS
class MyClassifier(torch.nn.Module):

    def __init__(self, basenet='bert-base-uncased', n_outputs=2, n_hidden=256):
        super(MyClassifier, self).__init__()
        self.encoder = AutoModel.from_pretrained( basenet,  output_hidden_states = False) 
       
        self.lstm  = torch.nn.LSTM(input_size= 768, hidden_size= 256)
        self.classifier = torch.nn.Linear(in_features= 256, out_features= 2) #fc


    def forward(self, input_id, attention_mask_id, token_type_id):
        
        embeddings = self.encoder(
            input_ids= input_id, 
            attention_mask= attention_mask_id, 
            token_type_ids= token_type_id)['last_hidden_state']
        
        # ic(embeddings)
        embeddings= embeddings.squeeze().permute(1, 0, 2)
        
        # ic(embeddings.shape) #torch.Size([128, 5, 768])
# "out" will give you access to all hidden states in the sequence
# "hidden" will allow you to continue the sequence and backpropagate,
# by passing it as an argument  to the lstm at a later time
# Add the extra 2nd dimension
        lstm_out, lstm_hidden = self.lstm(embeddings)
        # ic(lstm_hidden[0].shape)  #torch.Size([1, 5, 256]) #this is printed numerous times, because of back propogation or no. of lstm units

        embeddings    = lstm_hidden[0].squeeze()
        out = self.classifier (embeddings)											
        # ic(out.shape)
        
        return out

In [ ]:
# TRAINING FUNCTION
def train(model, 
          dl, 
          optimizer, 
          device,
          criterion = torch.nn.CrossEntropyLoss()#torch.nn.BCELoss()#torch.nn.CrossEntropyLoss()
          ):
    model.train()
    loss_avg = AverageMeter()
    
    for _, batch in enumerate(dl): #iterate in batch
      # ic(batch['input_id'].shape) #[32, 1, 128] (batch size, [one sentence,max-length of input])
      targets = batch['true_label'].to(device)
      input_id   = batch['input_id'].to(device).squeeze()
      attention_mask_id  = batch['attention_mask_id'].to(device).squeeze()
      segment_id = batch['segment_id'].to(device).squeeze()

      outputs = model(input_id, attention_mask_id, segment_id) # [32, 2] [batch size, model n_outputs]
      t = targets.float().view(-1,1) #[32,1]

      # ic(outputs[:, 0:2].shape, t[:, 0].shape) #outputs - first 2 cols of all rows
      loss = criterion(outputs[:, 0:2], t[:, 0].long())
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      loss_avg.update(loss.item(), targets.shape[0]) #[32,1] batch size
    
    return loss_avg.avg


# TESTING/EVALUATION FUNCTION

from sklearn.metrics import f1_score, accuracy_score

def test(model, 
         dl, 
         device, 
         criterion = torch.nn.CrossEntropyLoss()):
    model.eval()
    loss_avg = AverageMeter()
    o_   = torch.tensor([]).to(device) #use for metrics
    t_   = torch.tensor([]).to(device)

    with torch.no_grad():
        for _, batch in enumerate(dl):
            targets = batch['true_label'].to(device) #[128] batch size
            input_id   = batch['input_id'].to(device).squeeze()
            attention_mask_id  = batch['attention_mask_id'].to(device).squeeze()
            segment_id = batch['segment_id'].to(device).squeeze()

            outputs = model(input_id, attention_mask_id, segment_id)
            t = targets.float().view(-1,1) #[128,1] batch size
            loss = criterion(outputs[:, 0:2], t[:, 0].long())

            # EVALUATE
            max_o = torch.argmax(outputs[:, 0:2], dim=1).view(-1,1)
            # ic(max_o.shape) #[128,1]
            max_o = max_o.detach()
            o_    = torch.cat((o_, max_o)  , 0) #float and append
            t_    = torch.cat((t_, targets), 0)
            loss_avg.update(loss.item(), targets.shape[0])

    o_ = o_.cpu().numpy()
    t_ = t_.view(-1,1)# change shape same as o_
    t_ = t_.cpu().numpy()
    ic(t_.shape, o_.shape)
    fscore = f1_score(t_[:, 0], o_[:, 0])
    accuracy= accuracy_score(t_[:, 0], o_[:, 0])

    return loss_avg.avg, [fscore, accuracy]


In [ ]:
# LOAD DATASET 
train_dl = torch.utils.data.DataLoader(dataset= MyCustomDataset(
                          filename=  "data/train.csv",
                          basenet= BASENET),
                           batch_size= 32, shuffle= True, num_workers= 32, pin_memory= True)
val_dl = torch.utils.data.DataLoader(dataset= MyCustomDataset(
                          filename=  "data/dev.csv",
                          basenet= BASENET),
                           batch_size= 128, shuffle= True, num_workers= 32, pin_memory= True)
test_dl = torch.utils.data.DataLoader(dataset= MyCustomDataset(
                          filename=  "data/gold-test-27446.csv",
                          basenet= BASENET),
                           batch_size= 128, shuffle= True, num_workers= 32, pin_memory= True)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
# MODEL SELECTION
model = MyClassifier(BASENET, n_outputs=2) #experiment - bert with lstm

if torch.cuda.device_count() > 1:
	model = torch.nn.DataParallel(model)
 
model = model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning:

dropout option adds dropout after all but last recurrent layer, so non-zer

In [ ]:
# START TRAINING
learning_rate=2e-5 
start_epoch = 1
epochs = 10
optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)
train_loss_arr = []
val_loss_arr = []
metrics_arr = [] #[f1 , accuracy]

for ep in range(start_epoch, start_epoch+epochs):
  train_loss = train(model, train_dl, optimizer, device)
  val_loss, metrics = test(model, val_dl, device)
  train_loss_arr.append(train_loss)
  val_loss_arr.append(val_loss)
  metrics_arr.append(metrics)
  ic()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning:

This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.

ic| t_.shape: (1000, 1), o_.shape: (1000, 1)
ic| <ipython-input-25-a7ef25fa7b7f>:16 in <module> at 16:39:08.395
ic| t_.shape: (1000, 1), o_.shape: (1000, 1)
ic| <ipython-input-25-a7ef25fa7b7f>:16 in <module> at 16:42:28.210
ic| t_.shape: (1000, 1), o_.shape: (1000, 1)
ic| <ipython-input-25-a7ef25fa7b7f>:16 in <module> at 16:45:47.633
ic| t_.shape: (1000, 1), o_.shape: (1000, 1)
ic| <ipython-input-25-a7ef25fa7b7f>:16 in <module> at 16:49:07.227
ic| t_.shape: (1000, 1), o_.shape: (1000, 1)
ic| <ipython-input-25-a7ef25fa7b7f>:16 in <module> at 16:52:26.460


In [ ]:
# EVALUATE GOLD TEST
test_loss, test_metrics = test(model, test_dl, device)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
ic| lstm_hidden[0].shape: torch.Size([1, 128, 256])
ic| out.shape: torch.Size([128, 2])
ic| lstm_hidden[0].shape: torch.Size([1, 128, 256])
ic| out.shape: torch.Size([128, 2])
ic| lstm_hidden[0].shape: torch.Size([1, 128, 256])
ic| out.shape: torch.Size([128, 2])
ic| lstm_hidden[0].shape: torch.Size([1, 128, 256])
ic| out.shape: torch.Size([128, 2])
ic| lstm_hidden[0].shape: torch.Size([1, 128, 256])
ic| out.shape: torch.Size([128, 2])
ic| lstm_hidden[0].shape: torch.Size([1, 128, 256])
ic| out.shape: torch.Size([128, 2])
ic| lstm_hidden

In [ ]:
# REPORT EXPERIMENT RESULTS
print('-'*60)
print('train loss: ', end =" ") 
print('%.4f'%train_loss_arr[-1])
print('-'*60)

print(' val loss: ', end =" ")
print('%.4f'%val_loss_arr[-1], end =" " )      
print(' F1 score: ', end =" ")
print('%.2f'%(metrics_arr[-1][0]*100), end =" " )
print(' Accuracy: ', end =" ")
print('%.2f'%(metrics_arr[-1][1]*100))
print('-'*60)

print('test loss: ', end =" ")
print('%.4f'%test_loss, end =" " )      
print(' F1 score: ', end =" ")
print('%.2f'%(test_metrics[0]*100), end =" " )
print(' Accuracy: ', end =" ")
print('%.2f'%(test_metrics[1]*100) )
print('-'*60)


------------------------------------------------------------
train loss:  0.0079
------------------------------------------------------------
 val loss:  0.4953  F1 score:  91.93  Accuracy:  89.60
------------------------------------------------------------
test loss:  0.3229  F1 score:  92.82  Accuracy:  91.20
------------------------------------------------------------


In [ ]:
# SAVE MODEL
run_iter = 4 #iterations you ran the model
save_dir = os.path.join('my_model_weights', 'run_{}'.format(run_iter))

if not os.path.exists(save_dir):
  print("Creating directory {}".format(save_dir))
  os.makedirs(save_dir)

save_path = os.path.join(save_dir, "epoch_{}".format(epochs) + ".pth")
state = {
			'model': model.cpu().state_dict(),
			'optimizer': optimizer.state_dict(),
			'epoch': epochs,
		}

torch.save(state, save_path)
model = model.to(device)

Creating directory my_model_weights/run_4


In [ ]:
# LOAD SAVED MODEL

In [ ]:
# PLOT GRAPHS
import plotly.graph_objects as go

x_axis = list(range(1, len(train_loss_arr)+1)) #epochs
y_axis = list(train_loss_arr)

fig = go.Figure(data = go.Scatter(x= x_axis, y= y_axis))
fig.update_layout(title='train loss',
                xaxis_title='epochs',
                yaxis_title='loss')
fig.show(renderer="colab")

y2_axis = np.array(metrics_arr)
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x= x_axis, y= y2_axis[:,0],
                  mode='lines',
                  name='f1-score'))
fig2.add_trace(go.Scatter(x= x_axis, y= y2_axis[:,1],
                  mode='lines+markers',
                  name='accuracy'))
fig2.update_layout(title='performance',
                xaxis_title='epochs',
                yaxis_title='metrics')
fig2.show(renderer="colab")